In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url_base = "https://www.yourghoststories.com/real-ghost-story.php?story="

In [ ]:
from dataclasses import dataclass

@dataclass
class History:
  title: str
  place: str
  h_type: str
  history: str

In [ ]:
def get_page_history(num: str):
    response = requests.get(url_base + num)
    soup = BeautifulSoup(response.text, "html.parser")


    story_div = soup.find("div", id="story")
    title = soup.find("h1", class_="storytitle")
    storyinfo_divs = soup.find_all("div", class_="storyinfo")

    if len(storyinfo_divs) < 2:
        return None

    info_div = storyinfo_divs[1]


    country_tag = info_div.find("a", href=lambda x: x and "ghost-stories-countries" in x)
    country = country_tag.get_text(strip=True) if country_tag else None


    category_tag = info_div.find("a", href=lambda x: x and "ghost-stories-categories" in x)
    category = category_tag.get_text(strip=True) if category_tag else None

    paragraphs = [p.get_text() for p in story_div.find_all("p") or []]
    return History(
        title=title.get_text() if title else "",
        place=country,
        h_type=category,
        history= "".join(paragraphs)
    )


## Codigo optimizado con funciones asincronicas para sacarle mas rapidez

**Importante**
Despues de hacer el codigo, lo probe y ... no funciona xd, por que no tenemos permiso de hacerles tantas request asi xD

**Nota 2** Si se pudo ejecutar pero en script local

In [ ]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup

async def get_page_history(session, num: str):
    async with session.get(url_base + num) as response:
        html = await response.text()
        soup = BeautifulSoup(html, "html.parser")

        story_div = soup.find("div", id="story")
        title = soup.find("h1", class_="storytitle")
        storyinfo_divs = soup.find_all("div", class_="storyinfo")

        if len(storyinfo_divs) < 2:
            return None

        info_div = storyinfo_divs[1]
        country_tag = info_div.find("a", href=lambda x: x and "ghost-stories-countries" in x)
        country = country_tag.get_text(strip=True) if country_tag else None

        category_tag = info_div.find("a", href=lambda x: x and "ghost-stories-categories" in x)
        category = category_tag.get_text(strip=True) if category_tag else None

        paragraphs = [p.get_text() for p in story_div.find_all("p") or []]

        return History(
            title=title.get_text() if title else "",
            place=country,
            h_type=category,
            history="".join(paragraphs),
        )

async def main(nums):
    async with aiohttp.ClientSession() as session:
        tasks = [get_page_history(session, str(num)) for num in nums]
        results = await asyncio.gather(*tasks)
        return results
